In [2]:
import pandas as pd

## Общее описание:
Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

- date – дата совершения события

- event - событие

app_install – установка приложения
app_start – открыть приложения
register – зарегистрироваться 
search – перейти на страницу поиска товаров (каталог)
open_item – открыть товар
choose_item – отправить товар в корзину
tap_basket – перейти в корзину
purchase – подтверждение покупки
- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь

yandex-direct – Яндекс директ
google_ads – реклама в Google
vk_ads – реклама в ВК
instagram_ads – реклама в instagram
facebook_ads – реклама в facebook
referal – акция «приведи друга»
Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [3]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-v-tsuzoj/prod_analytics/KC_case_data .csv')

In [4]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   event         object 
 2   purchase_sum  float64
 3   os_name       object 
 4   device_id     int64  
 5   gender        object 
 6   city          object 
 7   utm_source    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 167.7+ MB


In [6]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month

### MAU февраля:

In [7]:
df.query("month == 2")['device_id'].nunique()

75032

### Количество установок в январе:

In [8]:
df.query("month == 1 and event == 'app_install'").shape[0]

80297

### Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

### Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [23]:
# Присвойте пользователям когорты по дню установки приложения
# и посчитайте для них  конверсию из установки в покупку в течение 7 дней.
# Для какой когорты конверсия была наибольшей? 


# Дата установки приложения
installs_users = df[['device_id','date','event']].query('event == "app_install"')

# Определение первой покупки у покупателя
first_purchase = df[['device_id','date','event']] \
                .query('event == "purchase"') \
                .groupby('device_id').agg({'date':'min'})

installs_users = installs_users.merge(first_purchase, how = 'left', on = 'device_id')
installs_users.rename(columns = {'date_x' : 'date_registration', 'date_y' : 'date_purchase'}, inplace = True )
installs_users['diff_registration_purchase'] = installs_users.date_purchase - installs_users.date_registration

#Функция для определения покупки в течение 7 дней(1 - да, 0 - нет)
def retention(row):
    if row['diff_registration_purchase'] <= pd.to_datetime('2024/01/08') - pd.to_datetime('2024/01/01'):
        val = 1
    else:
        val = 0
    return val

installs_users['retention'] = installs_users.apply (retention, axis=1)

In [25]:
ret = installs_users.groupby('date_registration',as_index = False).agg({'device_id':'count', 'retention': 'sum'})

ret['retent'] = ret.retention / ret.device_id * 100
ret.sort_values('retent', ascending = False)

,date_registration,device_id,retention,retent
0,2020-01-01,3579,1408,39.340598
8,2020-01-09,1424,558,39.185393
14,2020-01-15,4310,1650,38.283063
13,2020-01-14,5173,1973,38.140344
1,2020-01-02,3144,1186,37.722646
...,...,...,...,...
77,2020-03-18,1171,235,20.068318
88,2020-03-29,1117,223,19.964190
81,2020-03-22,1261,251,19.904837
87,2020-03-28,1091,209,19.156737


### С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [14]:
df.query("event == 'app_install'")['utm_source'].value_counts()

-                32460
yandex-direct    29368
google_ads       26286
vk_ads           23189
instagram_ads    20096
facebook_ads     13916
referal           9282
Name: utm_source, dtype: int64

### Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

In [15]:
new_df = df.copy()

In [16]:
reg_data = new_df[new_df['event'] == 'register']

In [17]:
new_df['registration_date'] = new_df['device_id'].map(reg_data.set_index('device_id')['date'])

In [18]:
voronka = new_df.query("event == 'search' or event == 'choose_item' or event == 'tap_basket'")

In [19]:
voronka_df = voronka.dropna(subset = ['registration_date'])

In [20]:
voronka_df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,registration_date
4445,2020-01-01,choose_item,NaN,ios,294193,female,Saint-Petersburg,instagram_ads,1,2020-01-01
4446,2020-01-01,choose_item,NaN,android,8658257,female,Saint-Petersburg,-,1,2020-01-01
4447,2020-01-01,choose_item,NaN,android,10345186,male,Saint-Petersburg,yandex-direct,1,2020-01-01
4448,2020-01-01,choose_item,NaN,ios,20209604,male,Moscow,facebook_ads,1,2020-01-01
4449,2020-01-01,choose_item,NaN,android,22449838,female,Moscow,google_ads,1,2020-01-01
...,...,...,...,...,...,...,...,...,...,...
2515054,2020-03-31,tap_basket,NaN,android,3043852,male,Moscow,-,3,2020-03-30
2515055,2020-03-31,tap_basket,NaN,android,20074065,female,Saint-Petersburg,facebook_ads,3,2020-03-31
2515056,2020-03-31,tap_basket,NaN,android,3162094,male,Moscow,referal,3,2020-01-13
2515057,2020-03-31,tap_basket,NaN,ios,8205,male,Saint-Petersburg,vk_ads,3,2020-01-01


In [21]:
voronka_df.query("event == 'search' or event == 'choose_item' or event == 'tap_basket'") \
  .groupby('event')['device_id'].nunique().sort_values(ascending = False)

event
tap_basket     78310
search         78310
choose_item    78310
Name: device_id, dtype: int64

In [22]:
CR_df = voronka_df.groupby(['event']).agg({'date' : 'count', 'device_id' : 'nunique'})

In [23]:
CR_df['device_id'] / CR_df['date']

event
choose_item    0.177789
search         0.138535
tap_basket     0.242982
dtype: float64

### Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

In [68]:
(df.query("event == 'purchase'").groupby('utm_source')['device_id'].nunique() / 
 df.query("event == 'app_start'").groupby('utm_source')['device_id'].nunique()) \
.sort_values()

utm_source
yandex-direct    0.295477
google_ads       0.297674
-                0.317556
instagram_ads    0.346658
facebook_ads     0.347382
vk_ads           0.362804
referal          0.399523
Name: device_id, dtype: float64

### Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [64]:
fp_median = df.query("event == 'purchase'") \
              .groupby(['device_id', 'utm_source'], as_index = False) \
              .agg({'purchase_sum' : 'first'})

fp_median.groupby('utm_source')['purchase_sum'].median().sort_values(ascending = False)

utm_source
-                398.5
referal          395.0
instagram_ads    393.5
vk_ads           393.0
yandex-direct    392.5
facebook_ads     390.5
google_ads       390.0
Name: purchase_sum, dtype: float64

---

Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

#### Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

In [133]:
revenue = df.query("event == 'purchase'").groupby('utm_source', as_index = False).agg({'purchase_sum' : 'sum'})
revenue

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [134]:
marketing_cost = {
    'utm_source': ['yandex-direct', 'google_ads', 'facebook_ads', 'instagram_ads', 'vk_ads'],
    'cost' : [10491707, 10534878, 8590498, 8561626, 9553531]
}

In [135]:
mc = pd.DataFrame(marketing_cost)
mc

,utm_source,cost
0,yandex-direct,10491707
1,google_ads,10534878
2,facebook_ads,8590498
3,instagram_ads,8561626
4,vk_ads,9553531


In [136]:
romi_df = revenue.merge(mc, on = 'utm_source')
romi_df['romi'] = (romi_df['purchase_sum'] - romi_df['cost']) / romi_df['cost']
romi_df.sort_values('romi', ascending = False)

,utm_source,purchase_sum,cost,romi
3,vk_ads,16389652.5,9553531,0.715560
2,instagram_ads,14546969.0,8561626,0.699090
0,facebook_ads,12249901.0,8590498,0.425983
4,yandex-direct,13915368.0,10491707,0.326321
1,google_ads,12868276.0,10534878,0.221493
